In [1]:
import os, itertools

import numpy as np
import pandas as pd

import scipy.stats as stats
import geopandas as gpd

from tqdm.notebook import tqdm

/home/shoerl/.mamba/envs/lead/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
fin_persons = "persons.csv"
fin_homes = "homes.gpkg"
output_path = "./output"

random_seed = 0
scaling = 1.0

delivery_days = 313 # 365

input_prefix = ""
output_prefix = ""

In [4]:
assert os.path.exists(fin_persons)
assert os.path.exists(fin_homes)

# Impute categories to population

In [5]:
df_persons = pd.read_csv(fin_persons, sep = ";")
df_persons = df_persons[df_persons["age"] >= 18]

In [6]:
# Age class

df_persons.loc[(df_persons["age"] >= 18) & (df_persons["age"] <= 34), "ac"] = 0 # 18 - 34
df_persons.loc[(df_persons["age"] >= 35) & (df_persons["age"] <= 49), "ac"] = 1 # 35 - 49
df_persons.loc[(df_persons["age"] >= 50) & (df_persons["age"] <= 64), "ac"] = 2 # 50 - 64
df_persons.loc[(df_persons["age"] >= 65), "ac"] = 3 # 65+
df_persons["ac"] = df_persons["ac"].astype(int)

In [7]:
# Socioprofessional class

df_persons.loc[df_persons["socioprofessional_class"].isin([1, 2]), "sc"] = 0 # CE,Artis,Com
df_persons.loc[df_persons["socioprofessional_class"].isin([3]), "sc"] = 1 # Cadre
df_persons.loc[df_persons["socioprofessional_class"].isin([4]), "sc"] = 2 # Prof Int
df_persons.loc[df_persons["socioprofessional_class"].isin([5]), "sc"] = 3 # Employe
df_persons.loc[df_persons["socioprofessional_class"].isin([6]), "sc"] = 4 # Ouvrier
df_persons.loc[df_persons["socioprofessional_class"].isin([7]), "sc"] = 5 # Retraite
df_persons.loc[df_persons["socioprofessional_class"].isin([8]), "sc"] = 6 # Sans Act
df_persons["sc"] = df_persons["sc"].astype(int)

In [8]:
# Household size class

df_household_size = df_persons.groupby("household_id").size().reset_index(name = "household_size")
df_persons = pd.merge(df_persons, df_household_size, on = "household_id")

df_persons.loc[df_persons["household_size"] == 1, "hc"] = 0 # 1
df_persons.loc[df_persons["household_size"] == 2, "hc"] = 1 # 2
df_persons.loc[df_persons["household_size"] == 3, "hc"] = 2 # 3
df_persons.loc[df_persons["household_size"] >= 4, "hc"] = 3 # 4+
df_persons["hc"] = df_persons["hc"].astype(int)

# Obtain marginals from report

In [9]:
# LAD by household size class and socioprofessional class (Figure 29)

marginal_hc_sc = np.array([
    [4, 12, 9, 14, 10, 5, 5],
    [27, 18, 15, 4, 16, 6, 13],
    [24, 26, 22, 6, 12, 10, 22],
    [30, 29, 22, 15, 29, 16, 17],
])

# LAD by age class and socioprofessional class (Figure 30)

marginal_ac_sc = np.array([
    [45, 29, 21, 18, 20, 0, 18],
    [30, 29, 19, 14, 22, 0, 11],
    [14, 17, 10, 5, 16, 12, 9],
    [12, 9, 0, 0, 0, 5, 2],
])

# LAD by socioprofessional class (Table 8)

marginal_sc = np.array([
    23.51, 21.19, 19.08, 15.15, 10.31, 9.77, 6.11
])

# Home delivery by socioprofessional class (Table 8)

probability_home_delivery = np.array([
    53.2, 46.8, 40.7, 49.1, 26.3, 56.5, 70.2
]) * 1e-2

# Total number of orders per year
marginal_total = 14.0

# Sample reference persons and aggregate counts

In [10]:
np.random.seed(random_seed)

sorter = np.arange(len(df_persons))
np.random.shuffle(sorter)

df_persons = df_persons.iloc[sorter]
df_persons = df_persons.drop_duplicates("household_id")

In [11]:
df = df_persons.groupby([
    "ac", "hc", "sc"
]).size().reset_index(name = "count")

df["weight"] = 1.0

# Apply weighting

In [12]:
for k in tqdm(range(100)): # Run 100 iterations
    
    # Weighting of household size x socioprofessional class
    for hc in range(4):
        for sc in range(7):
            f = (df["hc"] == hc) & (df["sc"] == sc)

            if np.count_nonzero(f) > 0:
                current_weight = (df[f]["weight"] * df[f]["count"]).sum()
                target_weight = marginal_hc_sc[hc, sc] * df[f]["count"].sum()

                if current_weight > 0:
                    factor = target_weight / current_weight
                    df.loc[f, "weight"] *= factor
                    
    # Weighting of household size x socioprofessional class
    for ac in range(4):
        for sc in range(7):
            f = (df["ac"] == ac) & (df["sc"] == sc)

            if np.count_nonzero(f) > 0:
                current_weight = (df[f]["weight"] * df[f]["count"]).sum()
                target_weight = marginal_ac_sc[ac, sc] * df[f]["count"].sum()

                if current_weight > 0:
                    factor = target_weight / current_weight
                    df.loc[f, "weight"] *= factor
    
    # Weighting of socioprofessional class
    for sc in range(7):
        f = df["sc"] == sc

        if np.count_nonzero(f) > 0:
            current_weight = (df[f]["weight"] * df[f]["count"]).sum()
            target_weight = marginal_sc[sc] * df[f]["count"].sum()

            if current_weight > 0:
                factor = target_weight / current_weight
                df.loc[f, "weight"] *= factor
                
    # Weighting of total       
    current_weight = (df["weight"] * df["count"]).sum()
    factor = marginal_total * df["count"].sum() / current_weight
    df["weight"] *= factor

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
(df["count"] * df["weight"]).sum() / df["count"].sum() # Should be around 14

14.0

# Generate deliveries

In [14]:
df_deliveries = df_persons[["household_id", "ac", "hc", "sc"]].copy()
df_deliveries = pd.merge(df_deliveries, df)

for sc, ac, hc in itertools.product(range(7), range(4), range(4)):
    f = df_deliveries["sc"] == sc
    f &= df_deliveries["ac"] == ac
    f &= df_deliveries["hc"] == hc
    
    if np.count_nonzero(f) > 0:
        weight = scaling * df_deliveries[f]["weight"].values[0] / delivery_days
        
        df_deliveries.loc[f, "orders"] = stats.poisson(weight).rvs(np.count_nonzero(f))
        df_deliveries.loc[f, "packages"] = stats.poisson(
            weight * probability_home_delivery[sc]).rvs(np.count_nonzero(f))

In [15]:
df_deliveries["orders"].sum() * delivery_days / len(df_deliveries) # Should be around 14

14.028868081171117

In [16]:
df_deliveries["packages"].value_counts()

0.0    93196
1.0     1932
2.0       28
4.0        1
Name: packages, dtype: int64

In [17]:
df_deliveries = df_deliveries[df_deliveries["packages"] > 0]
df_deliveries[["household_id", "packages"]]

,household_id,packages
160,32255,1.0
181,1372685,1.0
268,475680,1.0
273,1974169,1.0
364,770762,1.0
...,...,...
94810,55915,1.0
94816,152317,1.0
94831,1504676,1.0
94832,1609743,1.0


# Geographic information

In [18]:
df_spatial = gpd.read_file(fin_homes)

In [20]:
df_spatial = pd.merge(df_spatial, df_deliveries[["household_id", "packages"]], on = "household_id")
df_spatial.to_file(os.path.join(output_path, "parcels.gpkg"), driver = "GPKG")